In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/generated-dataset-nlp-project/newWords_top5.csv
/kaggle/input/generated-dataset-nlp-project/cleaned_generated.csv
/kaggle/input/nlp-assignment-1/hindi_corpus.txt


In [1]:
import re

In [2]:
input_path = '../input/nlp-assignment-1/hindi_corpus.txt'

In [4]:
#Filter word to remove non-Devanagari characters and punctuation marks
def filterWord(word):
    filterRegex = re.compile(r'[0-9।-ॱॽ,-:]')
    
    for char in word:
        char_unicode = ord(char)
        if(filterRegex.search(char) or char_unicode < 2300 or char_unicode > 2431):
            return False
            
    return True

In [14]:
#Find trigrams from given file path
def findTrigrams(input_path):
    linenumber = 1
    batchnumber = 1
    batchsize = 100_000 #process 100,000 lines and then write to output file, this prevents Thrashing
    batch = {}

    #Create a Regular Expression to filter out all words with any numeric character
    #or characters like ।, ॽ, :, -, ,
    filterRegex = re.compile(r'[0-9।-ॱॽ,-:]')
    
    with open(input_path, 'r', encoding='utf-8') as in_fp:
        for line in in_fp.readlines():
            tokens = []
            for token in line.split():
                if(filterWord(token)):
                    tokens.append(token)
                
            #Tri-gram of tokens
            for i in range(1, len(tokens) - 1):
                batch[tokens[i]] = tokens[i-1] + "_" + tokens[i+1]
            
            if(linenumber == batchsize):
                print(str(batchnumber) + " processed.")
                linenumber = 1
                batchnumber += 1
            else:
                linenumber += 1
            
    return batch

In [15]:
wordTrigrams = findTrigrams(input_path)

1 processed.
2 processed.
3 processed.
4 processed.
5 processed.
6 processed.
7 processed.
8 processed.
9 processed.
10 processed.
11 processed.
12 processed.
13 processed.
14 processed.
15 processed.
16 processed.
17 processed.
18 processed.
19 processed.
20 processed.
21 processed.
22 processed.
23 processed.
24 processed.
25 processed.
26 processed.
27 processed.
28 processed.
29 processed.
30 processed.
31 processed.
32 processed.
33 processed.
34 processed.
35 processed.
36 processed.
37 processed.
38 processed.
39 processed.
40 processed.
41 processed.
42 processed.
43 processed.
44 processed.
45 processed.
46 processed.
47 processed.
48 processed.
49 processed.
50 processed.
51 processed.
52 processed.
53 processed.
54 processed.
55 processed.
56 processed.
57 processed.
58 processed.
59 processed.
60 processed.
61 processed.
62 processed.
63 processed.
64 processed.
65 processed.
66 processed.
67 processed.
68 processed.
69 processed.
70 processed.
71 processed.
72 processed.
7

In [16]:
print(len(wordTrigrams))

1918803


In [17]:
#Read CSV file containing top 5 words mapped to each lemma
gendf = pd.read_csv('../input/generated-dataset-nlp-project/newWords_top5.csv')
gendf.head()

,S. No,Derived Word,Lemma
0,0,मोमबत्तियों,मोमबत्ती
1,1,मोमबत्तियाँ,मोमबत्ती
2,2,मोमबत्ति,मोमबत्ती
3,3,मोमबत्तिया,मोमबत्ती
4,4,गन्नाः,गन्ना


In [18]:
def findContext(df, wordTrigrams):
    context = {}
    notFound = {}
    
    for i in range(len(df)):
        data = df.iloc[i]
        word = data['Derived Word']
        lemma = data['Lemma']
        
        if word in wordTrigrams:
            context[word] = lemma + "_" + wordTrigrams[word]
        else:
            notFound[word] = lemma
            
        if(i % 1000 == 0):
            print("Till row " + str(i) + " processed.")
            
    return context, notFound

In [19]:
context, notFound = findContext(gendf, wordTrigrams)

Till row 0 processed.
Till row 1000 processed.
Till row 2000 processed.
Till row 3000 processed.
Till row 4000 processed.
Till row 5000 processed.
Till row 6000 processed.
Till row 7000 processed.
Till row 8000 processed.
Till row 9000 processed.
Till row 10000 processed.
Till row 11000 processed.
Till row 12000 processed.
Till row 13000 processed.
Till row 14000 processed.
Till row 15000 processed.
Till row 16000 processed.
Till row 17000 processed.
Till row 18000 processed.
Till row 19000 processed.
Till row 20000 processed.
Till row 21000 processed.
Till row 22000 processed.
Till row 23000 processed.
Till row 24000 processed.
Till row 25000 processed.
Till row 26000 processed.
Till row 27000 processed.
Till row 28000 processed.
Till row 29000 processed.
Till row 30000 processed.
Till row 31000 processed.
Till row 32000 processed.
Till row 33000 processed.
Till row 34000 processed.
Till row 35000 processed.
Till row 36000 processed.
Till row 37000 processed.
Till row 38000 processed.

In [20]:
print(len(context))

29741


In [21]:
print(len(notFound))

19717


In [22]:
print(context['मोमबत्तियों'])

मोमबत्ती_वह_की


In [23]:
'मोमबत्ती_वह_की'.split('_')

['मोमबत्ती', 'वह', 'की']

In [24]:
wordList = []
lemmaList = []
contextList = []

for word, values in context.items():
    values = values.split('_')
    wordList.append(word)
    lemmaList.append(values[0])
    contextList.append(values[1] + " " + word + " " + values[2])

In [25]:
print(len(wordList))
print(len(lemmaList))
print(len(contextList))

29741
29741
29741


In [26]:
contextdf = pd.DataFrame()
contextdf['Index'] = range(1, len(wordList) + 1)
contextdf['Lemma'] = lemmaList
contextdf['Derived Word'] = wordList
contextdf['Context'] = contextList
contextdf.head()

,Index,Lemma,Derived Word,Context
0,1,मोमबत्ती,मोमबत्तियों,वह मोमबत्तियों की
1,2,मोमबत्ती,मोमबत्तियाँ,है मोमबत्तियाँ कम
2,3,मोमबत्ती,मोमबत्तिया,तथा मोमबत्तिया जलाकर
3,4,गन्ना,उपगन्ना,उन्होंनें उपगन्ना आयुक्त
4,5,गन्ना,गन्नाधिकारी,जिला गन्नाधिकारी साहब


In [27]:
contextdf.to_csv('./Context.csv')

In [28]:
df = pd.read_csv('../input/generated-dataset-nlp-project/newWords_top5.csv')
df.head()

,S. No,Derived Word,Lemma
0,0,मोमबत्तियों,मोमबत्ती
1,1,मोमबत्तियाँ,मोमबत्ती
2,2,मोमबत्ति,मोमबत्ती
3,3,मोमबत्तिया,मोमबत्ती
4,4,गन्नाः,गन्ना


In [29]:
#Only take those words which have the lemma in their prefix
def cleanData(df):
    wordToLemmaMap = {} #Dictionary to store mapping from Derived Word to Lemma
    
    for i in range(len(df)):
        data = df.iloc[i]
        word = data['Derived Word']
        lemma = data['Lemma']
        lemmalen = len(lemma)
        suffixFound = False
        
        if(word == lemma):
            wordToLemmaMap[word] = lemma
        
        else:
            j = 0
            while(j < len(word)):
                if(j == lemmalen):
                    break
                if(word[j] != lemma[j]):
                    break
                j += 1
            
            if(j == lemmalen):
                wordToLemmaMap[word] = lemma

    return wordToLemmaMap

In [30]:
wordToLemmaMap = cleanData(df)
print(len(wordToLemmaMap))

16240


In [31]:
wordToLemmadf = pd.DataFrame(wordToLemmaMap.items())
wordToLemmadf.head()

,0,1
0,गन्नाः,गन्ना
1,गन्नाधिकारी,गन्ना
2,संदेहलाभ,संदेह
3,संदेह्,संदेह
4,संदेहो,संदेह


In [32]:
wordToLemmadf.to_csv('./cleaned_generated.csv')

In [33]:
cleangendf = pd.read_csv('../input/generated-dataset-nlp-project/cleaned_generated.csv')
cleangendf.head()

,Unnamed: 0,Derived Word,Lemma
0,1,गन्नाः,गन्ना
1,2,गन्नाधिकारी,गन्ना
2,3,संदेहलाभ,संदेह
3,4,संदेह्,संदेह
4,5,संदेहो,संदेह


In [34]:
context, notFound = findContext(cleangendf, wordTrigrams)

Till row 0 processed.
Till row 1000 processed.
Till row 2000 processed.
Till row 3000 processed.
Till row 4000 processed.
Till row 5000 processed.
Till row 6000 processed.
Till row 7000 processed.
Till row 8000 processed.
Till row 9000 processed.
Till row 10000 processed.
Till row 11000 processed.
Till row 12000 processed.
Till row 13000 processed.
Till row 14000 processed.
Till row 15000 processed.
Till row 16000 processed.


In [35]:
print(len(context))
print(len(notFound))

9345
6895


In [36]:
wordList = []
lemmaList = []
contextList = []

for word, values in context.items():
    values = values.split('_')
    wordList.append(word)
    lemmaList.append(values[0])
    contextList.append(values[1] + " " + word + " " + values[2])

In [37]:
print(len(wordList))
print(len(lemmaList))
print(len(contextList))

9345
9345
9345


In [38]:
contextdf = pd.DataFrame()
contextdf['Index'] = range(1, len(wordList) + 1)
contextdf['Lemma'] = lemmaList
contextdf['Derived Word'] = wordList
contextdf['Context'] = contextList
contextdf.head()

,Index,Lemma,Derived Word,Context
0,1,गन्ना,गन्नाधिकारी,जिला गन्नाधिकारी साहब
1,2,संदेह,संदेहलाभ,में संदेहलाभ दिया
2,3,संदेह,संदेहो,गंभीर संदेहो के
3,4,मारपीट,मारपीटकर,जाकर मारपीटकर और
4,5,मारपीट,मारपीटा,को मारपीटा और


In [39]:
contextdf.to_csv('./Context_CleanedData.csv')